# Let in LINQ 💾

- if we want to store the result of a subexpression in order to use it in subsequent clauses
- https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/keywords/let-clause

In [ ]:
using static System.Console;

string[] strings =
[
    "A penny saved is a penny earned",
    "The early bird catches the worm",
    "The pen is mightier than the sword"
];
char[] vovels = [ 'a', 'e', 'i', 'o', 'u' ];

var query = from sentence in strings
            let words = sentence.Split(' ')
            
            from word in words
            let w = word.ToLower()
            
            where vovels.Contains(w[0])
            select word;

foreach (var v in query)
    WriteLine($"{v} starts with a vowel");

# Closure 🎗️

<img src=images/functional-programming-meme-3.jpg>

## Revision Closure 101

In [ ]:
using static System.Console;

int outerVariable = 10;

Action displayValue = () => WriteLine(outerVariable);
Func<int, int> addValue = (x) => x + outerVariable;

outerVariable = 15;

displayValue();
WriteLine(addValue(5));

- treating code as data
- dynamic nature of C# compiler
- https://learn.microsoft.com/en-us/dotnet/api/system.delegate

In [ ]:
using static System.Console;

int outerVariable = 10;

int _captureDeclareTimeValue = outerVariable;
Action displayValue = () => WriteLine(_captureDeclareTimeValue);
Func<int, int> addValue = (x) => x + _captureDeclareTimeValue;

outerVariable = 15;

displayValue();
WriteLine(addValue(5));

## Local Functions

<img src=images/functional-programming-meme-2.jpg>

In [ ]:
#nullable enable

void Process(string?[] lines, string mark)
{
    foreach (var line in lines)
    {
        if (IsValid(line))
        {
            // Processing logic...
        }
    }

    bool IsValid(string? line) =>
        !string.IsNullOrEmpty(line) && line.Length >= mark.Length;
}

Its matter of choice; how you can express better

In [ ]:
using System.Linq;

int Factorial(int n)
{
    if (n < 0) throw new ArgumentException("Input must be a non-negative integer");

    return nthFactorial(n);

    int nthFactorial(int n) =>
        n < 2 ? 1 : n * nthFactorial(n - 1);
}

Factorial(5)

In [ ]:
using System.Linq;

int Factorial(int n)
{
    if (n < 0) throw new ArgumentException("Input must be a non-negative integer");

    Func<int, int> nthFactorial = default(Func<int, int>); // for recursion we had to define it
    nthFactorial = n => n < 2 ? 1 : n * nthFactorial(n - 1);

    return nthFactorial(n);
}

Factorial(5)

In [ ]:
IEnumerable<string> sequenceToLowercase(IEnumerable<string> input)
{
    if (null == input) return Enumerable.Empty<string>();   // throw new ArgumentException("input is not valid");
    if (!input.Any()) return Enumerable.Empty<string>();    // throw new ArgumentException("There are no items to convert to lowercase.");
    
    return LowercaseIterator();
    
    IEnumerable<string> LowercaseIterator()
    {
        foreach (var output in input.Select(item => item.ToLower()))
            yield return output; // this is not possible in lambdas
    }
}

void print<T>(IEnumerable<T> input)
{
    Console.WriteLine(string.Join(", ", input));
}

print(sequenceToLowercase(null));
print(sequenceToLowercase(new string[] { }));
print(sequenceToLowercase(new [] { "Hello", "World" }));

- https://learn.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/local-functions

# Functors ⛈️

- https://en.wikipedia.org/wiki/Functor_(functional_programming)
- A functor is a design pattern that allows one to apply a function to values inside a generic type without changing the structure of the generic type

- Map of Javascript
- Select() of C#
- Projection

In [ ]:
// javascript

const numbers = [1, 2, 3, 4];
const functor = numbers.map(x => x * 2);        // array type has map implementation
console.log(functor);


class Functor {
  constructor(value) {
    this.value = value;
  }

  map(fn) {                                     // our custom type implementing a map function
    return new Functor(fn(this.value));         // we are generating a new instance 👈
  }
}

const customFunctor = new Functor(5);
const mappedFunctor = customFunctor.map(x => x * 2);
console.log(mappedFunctor);

In [ ]:
using static System.Console;
using System.Linq;

// Java like Generic / Seperate Map function
// But its an extension method and can be brought into context on need basis
static IEnumerable<TResult> Map<T, TResult>(this IEnumerable<T> source, Func<T, TResult> selector) =>
    source.Select(selector); // I am cheating here; and using LINQ's select

var numbers = new List<int> { 1, 2, 3, 4, 5 };
var squaredNumbers = numbers.Map(x => x * x);
foreach(var number in squaredNumbers)
    WriteLine(number);

foreach(var number in numbers.Select(n => n * n))
    WriteLine(number);

In [ ]:
using static System.Console;
using System.Linq;

record Student(string Name, double LastScore, double TotalScore);

var students = new[] { new Student("Khurram", 80, 110), new Student("Abdullah", 90, 120) };

foreach(var student in students.Select(s => new         // generating a new type / anonymous type
    {
        s.Name, Percentage = s.LastScore / s.TotalScore * 100
    }))

    WriteLine($"Name: {student.Name}, Percentage: {student.Percentage:0.00}");

In [ ]:
// Common Types; IEnumerable<T> and Linq / Functor

using static System.Console;
using System.Linq;

//Func<string, bool>
var validate = s => !string.IsNullOrEmpty(s)
    && s.All(char.IsDigit)
    && s.Length > 4;

string t1 = "1234567";
string f1 = "1234";
string f2 = "123abc";
string f3 = null;

WriteLine(validate(t1));
WriteLine(validate(f1));
WriteLine(validate(f2));
WriteLine(validate(f3));

In [ ]:
using static System.Console;

class Functor
{
    readonly Func<int, int> function;
    public Functor(Func<int, int> function) => (this.function) = function;
    public int Invoke(int x) => this.function(x);               // its not necessary to call it Map
    public int this[int x] => this.function(x);                 // indexer
}

var square = new Functor(x => x * x);
WriteLine(square.Invoke(5));    //OOP way
WriteLine(square[5]);

# Referential Transparency 📝

- Referential transparency is a property of expressions in which an expression can be replaced with its corresponding value without changing the program's behavior
- This property is key to functional programming as it makes reasoning about code easier and allows for optimizations such as memoization.

In [ ]:
int Factorial(int n)
{
    if (n < 0)                              // recursion termination
        throw new ArgumentException("Input must be a non-negative integer", nameof(n));
    else if (n == 0)                        // recursion termination
        return 1;
    else
        return n * Factorial(n - 1);        // tail recursion
}

Factorial(5)

## Memoization

In [ ]:
using static System.Console;
using System.Collections.Generic;

static Dictionary<int, long> memo = new Dictionary<int, long>();

// Memoized Fibonacci function
static long Fibonacci(int n)
{
    if (n <= 1) return n;

    if (memo.ContainsKey(n)) return memo[n];

    long result = Fibonacci(n - 1) + Fibonacci(n - 2);
    memo[n] = result;
    return result;
}

for (int i = 0; i <= 40; i++)
    WriteLine($"Fibonacci({i}) = {Fibonacci(i)}");

In [ ]:
using static System.Console;

class Functor<T>
{
    Dictionary<T, T> dictionary = new();
    Func<T, T> function;
    
    public Functor(Func<T, T> function) => (this.function) = function;
    
    T hitOrMiss(T x)
    {
        if (this.dictionary.ContainsKey(x)) return this.dictionary[x];

        T y = this.function(x);
        this.dictionary.Add(x, y);
        return y;
    }

    public T Invoke(T x) => this.hitOrMiss(x);
    public T this[T x] => this.hitOrMiss(x); // indexer
}

var square = new Functor<int>(x => x * x);
WriteLine(square[5]);

- https://learn.microsoft.com/en-us/dotnet/api/microsoft.extensions.caching.memory.imemorycache

# Currying 🎊

How we can pass functions to function; we can return a function from a function as well

In [ ]:
using static System.Console;

int traditionalAdd(int a, int b) => a + b;

Func<int, int> elaboratedAddX(int x) // returning a function that takes a parameter
{
    return new Func<int, int>(
        (y) =>
        {
            return x + y;
        }
    );
}

Func<int, int> addX(int x) => y => x + y;

Func<int, int, int> GiveMeOperations(bool adding)
{
    return adding ?
        (a, b) => a + b :
        (a, b) => a - b;
}

var add1 = elaboratedAddX(5);
var add2 = addX(5);
var op = GiveMeOperations(adding: true);

WriteLine(add1(4));
WriteLine(add2(4));
WriteLine(op(5, 4));

__Lets make things interesting__

In [ ]:
using static System.Console;

Func<int, Func<int, int>> elaboratedCurriedAdd() // returning a function that takes a function as parameter
{
    return new Func<int, Func<int, int>>(
        (x) =>
        {
            return new Func<int, int>(
                (y) =>
                {
                    return x + y;
                }
            );
        }
    );
}
Func<int, Func<int, int>> curriedAdd() => x => y => x + y;

var curry = elaboratedCurriedAdd();     // its a function
var adder4 = curry(4);                  // its still a function
var r2 = adder4(5);                     // we have the final answer

var r3 = curriedAdd()(4)(5);            // let r3 = curriedAdd 4 5

WriteLine(r2);
WriteLine(r3);

Haskell Curry was a logician and mathematician whose work significantly influenced the field of functional programming. Curry functions, or "currying," refer to the process of transforming a function that takes multiple arguments into a series of functions that each take a single argument.
What is Currying?

Currying is a technique used in functional programming where a function with multiple parameters is transformed into a sequence of functions, each taking a single parameter. It allows for more flexible function composition and partial application of functions.

- Currying is useful for partial implementations
- Function Composition
- Reusability
- Declarative Code
- Immutability
- Higher Order Functions

# Putting it Altogether 🎇

In [ ]:
using static System.Console;
using System.Threading;

record Device(string ip);

static void DoSomething(this Device device) { Thread.Sleep(3000); }
static void DoSomethingElse(this Device device) { Thread.Sleep(3000); }

void log(Device d, string message) { WriteLine($"[{d.ip}] {message}"); }
void error(Device d, Exception ex) { WriteLine($"[{d.ip}] {ex}"); }

void Process(CancellationToken token, IEnumerable<Device> devices)
{
    Parallel.ForEach(devices, d =>
    {
        try
        {
            if (token.IsCancellationRequested) return;
            log(d, "Doing something");
            d.DoSomething();

            if (token.IsCancellationRequested) return;
            log(d, "Doing something else");
            d.DoSomethingElse();
        }
        catch (Exception ex)
        {
            error(d, ex);
        }
    });
}

In [ ]:
using static System.Console;
using System.Threading;

record Device(string ip);

static void DoSomething(this Device device) { Thread.Sleep(3000); }
static void DoSomethingElse(this Device device) { Thread.Sleep(3000); }

Action<Device, CancellationToken, Action<Device, string>> getStep(Action<Device> func, string step)
{
    return (d, token, log) =>
    {
        if (token.IsCancellationRequested) return;
        log(d, step);
        func(d);
    };
}

Action<Action<Device, string>, Action<Device, Exception>> processDevice(CancellationToken token, Device d)
{
    return (log, error) =>
    {
        try
        {
            getStep(d => d.DoSomething(), "Doing something")(d, token, log);
            getStep(d => d.DoSomethingElse(), "Doing something else")(d, token, log);
        }
        catch (Exception ex)
        {
            error(d, ex);
        }
    };
}

void Process(CancellationToken token, Func<IEnumerable<Device>> getDevices)
{
    Parallel.ForEach(getDevices(), d => processDevice(token, d)
    (
        (d, s) => WriteLine($"[{d.ip}] {s}"),
        (d, e) => WriteLine($"[{d.ip}] {{e}}")
    ));
}

Process(CancellationToken.None, () => new [] { new Device("1"), new Device("2") });

In [ ]:
// NgRx Examples - Disclaimer: I am not NgRx user/expert :)

import { createAction, createSelector, createReducer, props, on, Action } from '@ngrx/store';

// Curried Action Creator
const createCustomAction = (type: string) => (payload?: any) =>
  createAction(type, props<{ payload: any }>());

const loadItems = createCustomAction('[Items] Load Items');
const addItem = createCustomAction('[Items] Add Item');
store.dispatch(loadItems({ payload: { category: 'books' } }));
store.dispatch(addItem({ payload: { id: 1, name: 'NgRx Book' } }));

// Curried Selector
const selectFeature = (featureName: string) => createSelector(
  (state: any) => state[featureName],
  (featureState) => featureState
);

const selectItemsFeature = selectFeature('items');
const items = store.select(selectItemsFeature);

//Curried State Updaters
const updateState = (key: string) => (state: any, payload: any) => ({
  ...state,
  [key]: payload,
});

const itemsReducer = createReducer(
  initialState,
  on(loadItemsSuccess, (state, { payload }) => updateState('items')(state, payload)),
  on(addItemSuccess, (state, { payload }) => updateState('items')([...state.items, payload]))
);

function reducer(state: any, action: Action) {
  return itemsReducer(state, action);
}

In [ ]:
//Redux Example - Disclaimer: I am not Redux user/expert :)

import { createStore, applyMiddleware } from 'redux';
import thunk from 'redux-thunk';
import logger from 'redux-logger';
import rootReducer from './reducers';

const store = createStore(
  rootReducer,                    
  applyMiddleware(thunk, logger)  // the createStore is expecting a function here
);                                // we provided what applyMiddleware returned which is a curried function

# Immutable Data Types 🗄️

- Principles and goals of Immutable Data Types and Functional Progamming aligs
- Referential Transparency; *expression can be replaced with value*
- Thread Safety / Concurrency
    - This is the reason Functional Programming is Cloud Friendly
- Predictable Behavior
- No Side Effect
    - Pure Functions

Haskell: all data is immutable by default
Java/C# and even F#: all data is mutable by default

In [ ]:
let list = [1; 2; 3]
let newList = 0 :: list  -- Prepend 0 to the list, creating a new list

## Immutable Data Types (C#)

Built in Immutable Data Types
- strings
- System.DateTime and System.DateTimeOffset
- Tuples

In [ ]:
var tuple = Tuple.Create(1, "hello", true);
var valueTuple = (1, "hello", true);
//tuple.Item1 = 2;        //  will not work
valueTuple.Item1 = 2;

Immutable Collections of System.Collections.Immutable
- ImmutableList, ImmutableArray
- ImmutableDictionary
- ImmutableQueue, ImmutableStack
- ImmutableSortedDictionary, ImmutableSortedSet

## Custom Immutable Types & Deconstruction ✂️

Custom Immutable Types
- Classes
- Records

In [ ]:
class Person
{
    public string FirstName { get; }
    public string LastName { get; }

    public Person(string firstName, string lastName) =>
        (FirstName, LastName) = (firstName, lastName);
    
    public void Deconstruct(out string firstName, out string lastName)  // by adding this method; class/instances become deconstructable
    {
        firstName = this.FirstName;
        lastName = this.LastName;
    }
}

var person = new Person("Khurram", "Az");
Person correctedPerson = new Person(person.FirstName, "Aziz");
var (firstName, lastName) = correctedPerson;     // deconstruction to tuple
Console.WriteLine($"Nice to meet you {firstName}");

In [ ]:
record Person(string FirstName, string LastName);
var person = new Person("Khurram", "Az");
Person correctedPerson = person with { LastName = "Aziz" };
var (firstName, lastName) = correctedPerson;     // deconstruction to tuple
Console.WriteLine($"Nice to meet you {firstName}");

# Pattern Matching 🧱

## Usages in C#

- __is expression__
    - if (x is Person p)
- __switch statement__
    - switch (x) { case Person p: }
- __switch expression__
    - the cool cousin of switch statement
    - we will learn about it shortly

## C# Pattern Matching Journey

In [ ]:
record Shape(double Area) {}
record Circle(int Radius) : Shape(Math.PI * Radius * Radius) {}
record Rectangle(int Height, int Width) : Shape (Height * Width) { }

var randomShape = new Shape(100);

// C# 6
if (randomShape is Circle)
{
    Circle c = randomShape as Circle;
}

// C# 7
if (randomShape is Circle c)
{
}
switch (randomShape)
{
    case Circle c:
        break;
    case Rectangle r when r.Height == r.Width:
        break;
    case Rectangle r:
        break;
}

// C# 8
if (randomShape is Circle { Radius: 10}) // we can have checks across multiple properties
{
}

var detail = randomShape switch
{
    Circle c => $"Its Circle with Radius {c.Radius}",
    Rectangle r when r.Height == r.Width => $"Its a square with side {r.Height}",
    { Area: 100 } => $"It can be either circle or rectangle or some shape with Area 100",
    _ => "We dont know about it yet"
};

// C# 9
if (randomShape is Circle c { Radius: > 10 and < 20})
{}
if (randomShape is not Circle) // or is not null
{}
detail = randomShape switch
{
    Circle { Area: > 100 } c => $"Its a large Circle with Radius {c.Radius}",
    Circle c => $"Its Circle with Radius {c.Radius}",
    Rectangle r when r.Height == r.Width => $"Its a square with side {r.Height}",
    { Area: 100 } => $"It can be either circle or rectangle or some shape with Area 100",
    _ => "We dont know about it yet"
};

var areaDetail = randomShape.Area switch // var storyPoints = 50;
{
    > 100 and < 1000 => "",
    > 100 => "Its shape that will take more than 4 things",
    _ => "Its fine"
};

In [ ]:
using System.Linq;

// C# 10
record Order(int Id, string Product, int Amount);
record Customer(string Name, List<Order> Orders);

var customers = new List<Customer>
{
    new Customer("Alpha",
        Orders: new List<Order>
        {
            new Order(1, "Book", 20),
            new Order(2, "Pen", 30)
        }),
    new Customer("Bravo",
        Orders: new List<Order>
        {
            new Order(3, "Notebook", 10)
        })
};

foreach(var customer in customers)
{
    if (customer is { Orders: { Count: > 1 }}  // we have a repeated order
        && customer.Orders.Sum(o => o.Amount) >= 50) // we have earned good

        Console.WriteLine($"{customer.Name} is a good customer");
}

In [ ]:
using System.Linq;

static bool isAlphabetOrSpace(char c) =>    // Functional Programming sorcery
    c is (>= 'a' and <= 'z') or
    (>= 'A' and <= 'Z') or
    ' ';
    
static bool IsClean(this string s) =>       // extension methods allow us to glue our functional stuff onto OOP stuff
    !string.IsNullOrWhiteSpace(s) &&        // lets not forget null checks
    s.All(isAlphabetOrSpace) &&             // Linq-y
    s.StartsWith(" ") && s.EndsWith(" ");   // OOP-ish

#nullable enable

record Application(string? Manufacturer, string? Name);       // immutable data type

var app = new Application(null, "Visual Studio");
if (!app.Manufacturer.IsClean() || !app.Name.IsClean())      // business validation
    Console.WriteLine("We dont have clean values");

#nullable disable

## Checking for Null

In [ ]:
class Something {}
Something something = null;
Something another = new Something();

if (something is null) Console.WriteLine("something is null");
if (another is not null) Console.WriteLine("another is not null");

## Type Testing

In [ ]:
class Something {}
Something something = new Something();      // something is not null

var disposable = something as IDisposable;  // but disposeable will be null as something is not IDisposable
if (null == disposable) Console.WriteLine("disposable is null");

In [ ]:
IEnumerable<int> numbers = [10, 20, 30];
IList<int> indexable = numbers as IList<int>;
if (indexable != null)
    Console.WriteLine(indexable[0] + indexable[indexable.Count - 1]);  // output: 40

In [ ]:
int i = 23;
object iBoxed = i;
int? nullable = 7;

if (iBoxed is int a && nullable is int b)
    Console.WriteLine(a + b);

In [ ]:
int i = 5;
Console.WriteLine(typeof(int));
Console.WriteLine(i.GetType() == typeof(int));

In [ ]:
int? maybe = 12;

if (maybe is int number)
    Console.WriteLine($"The nullable int 'maybe' has the value {number}");
else
    Console.WriteLine("The nullable int 'maybe' doesn't hold a value");

In [ ]:
T MidPoint<T>(IEnumerable<T> sequence)
{
    if (sequence is IList<T> list)
        return list[list.Count / 2];
    else if (sequence is null)
        throw new ArgumentNullException(nameof(sequence), "Sequence can't be null.");
    else
    {
        int halfLength = sequence.Count() / 2 - 1;
        if (halfLength < 0) halfLength = 0;
        return sequence.Skip(halfLength).First();
    }
}

MidPoint([1, 2, 3, 4])

## Compare discrete values

In [ ]:
enum Operation { SystemTest, Start, Stop, Reset, New }
enum State { Started, Completed, Failed, NotAvailable }

State PerformOperation(Operation command) =>
   command switch
   {
       Operation.SystemTest => State.Completed,
       Operation.Start => State.Started,
       Operation.Stop => State.Completed,
       Operation.Reset => State.NotAvailable,
       _ => throw new ArgumentException("Invalid enum value for command", nameof(command)),
   };

PerformOperation(Operation.SystemTest)

## Relational patterns

In [ ]:
string WaterState(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        (> 32) and (< 212) => "liquid",
        < 32 => "solid",
        > 212 => "gas",
        32 => "solid/liquid transition",
        212 => "liquid / gas transition",
    };

// if not all cases are handled; we get compiler warning

string WaterState2(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        < 32 => "solid",
        32 => "solid/liquid transition",
        < 212 => "liquid",
        212 => "liquid / gas transition"
        //_ => "gas",
};

Console.WriteLine(WaterState(213));
Console.WriteLine(WaterState2(213));

In [ ]:
record Order(int Items, decimal Cost); // why we are using decimal ? :)

decimal CalculateDiscount(Order order) =>
    order switch
    {
        { Items: > 10, Cost: > 1000.00m } => 0.10m,
        { Items: > 5, Cost: > 500.00m } => 0.05m,
        { Cost: > 250.00m } => 0.02m,
        null => throw new ArgumentNullException(nameof(order), "Can't calculate discount on null order"),
        var someObject => 0m,
    };

In [ ]:
record Order(int Items, decimal Cost);

// deconstruction and pattern matching
decimal CalculateDiscount(Order order) =>
    order switch
    {
        ( > 10,  > 1000.00m) => 0.10m,
        ( > 5, > 50.00m) => 0.05m,
        { Cost: > 250.00m } => 0.02m,
        null => throw new ArgumentNullException(nameof(order), "Can't calculate discount on null order"),
        var someObject => 0m,
    };

## List Patterns

In [ ]:
using static System.Console;

int[] numbers = { 1, 2, 3 };

WriteLine(numbers is [1, 2, 3]);            // True
WriteLine(numbers is [1, 2, 4]);            // False
WriteLine(numbers is [0 or 1, <= 2, >= 3]); // True

In [ ]:
using static System.Console;

WriteLine(new[] { 1, 2, 3, 4, 5 } is [> 0, > 0, ..]);
WriteLine(new[] { 1, 1 } is [_, _, ..]);

WriteLine(new[] { 1, 2, 3, 4 } is [.., > 0, > 0]);
WriteLine(new[] { 2, 4 } is [.., 2, 4]);

WriteLine(new[] { 1, 2, 3, 4 } is [>= 0, .., 2 or 4]);
WriteLine(new[] { 1, 0, 0, 1 } is [1, 0, .., 0, 1]);

In [ ]:
using static System.Console;

void Validate(int[] numbers) // we cant use IEnumerable; as we will not be able to use Length
{
    var result = numbers is [< 0, .. { Length: 2 or 4 }, > 0] ? "valid" : "not valid";
    WriteLine(result);
}

Validate(new[] { -1, 0, 1 });
Validate(new[] { -1, 0, 0, 1 });

https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/proposals/csharp-11.0/list-patterns

## Resources

- https://learn.microsoft.com/en-us/dotnet/csharp/fundamentals/functional/pattern-matching
- https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/operators/patterns 👈
- https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/language-specification/patterns
- https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/proposals/csharp-8.0/patterns 👈

# Pattern Matching and Deconstruction 🧱✂️

In [ ]:
using static System.Console; // to get rid of class name noise

// records give us one liner types and they have all the OOP goodness
abstract record WeatherResponse;
record Success(string City, double Temperature, string Condition) : WeatherResponse;
record Sunny(string City, double Temperature) : Success(City, Temperature, "Sunny");
record NotFound(string City) : WeatherResponse;
record Error(string ErrorMessage) : WeatherResponse;

WeatherResponse GetWeatherResponse(string city) => city switch
{
    // Simulating different responses
    "Lahore" => new Sunny("Lahore", 49.0),
    "Faisalabad" => new Success("Faisalabad", 49.5, "Very Sunny"),
    not null => new NotFound(city),
    _ => new Error("Unknown error occurred")
};

void HandleWeatherResponse(WeatherResponse response)
{
    Func<string, bool> printAndReturn = s => // so we can use it with one-liner =>s in pattern matching
    {
        WriteLine(s);
        return true;
    };

    // discarding returned value; we are only interested in console writing
    _ = response switch // switch pattern matching work with returns
    {
        //some OOP spices
        Success(string city, double temperature, string condition) => // will match Success and Sunny
            printAndReturn($"Weather in {city}: {temperature}°C, {condition}"),
        NotFound(string city) =>
            printAndReturn($"'{city}' city not found."),
        Error(string errorMessage) =>
            printAndReturn($"Error: {errorMessage}"),
        _ => printAndReturn("Unknown response type.")
    };
}

// back to imperative
var cities = new[] { "Lahore", "Gotham", "Faisalabad", null };
foreach (var city in cities)
    HandleWeatherResponse(GetWeatherResponse(city));

# Pattern Matching and Recursion 🧱🔮

In [ ]:
// Elixir
defmodule Factorial do
  def of(n) when n < 0, do: "undefined"
  def of(0), do: 1
  def of(n), do: n * of(n - 1)
end

Factorial.of 5

<img src="images/erlang-functions.png">

In [ ]:
// Haskell

size :: Int -> String
size 1 = "Small"
size 2 = "Medium"
size 3 = "Large"
size _ = "Unknown size"

In [ ]:
// Prolog

% Facts: Symptoms
has_symptom(flu, fever).
has_symptom(flu, headache).
has_symptom(flu, body_aches).
has_symptom(flu, cough).
has_symptom(flu, sore_throat).
has_symptom(flu, runny_nose).
has_symptom(allergy, sneezing).
has_symptom(allergy, watery_eyes).
has_symptom(allergy, runny_nose).
has_symptom(allergy, itchy_eyes).
has_symptom(cold, sneezing).
has_symptom(cold, watery_eyes).
has_symptom(cold, runny_nose).
has_symptom(cold, cough).
has_symptom(cold, sore_throat).

% Determine illness (predicate function)
diagnose_illness(Symptom, Illness) :-
    has_symptom(Illness, Symptom).

% Querying illness (query function)
?- diagnose_illness(fever, What).

In [ ]:
#!connect jupyter --kernel-name pythonkernel --kernel-spec python3

In [ ]:
def factorial(n):
    match n:
        #case _ if n < 0:
        #    raise ValueError("Input must be a non-negative integer")
        case 0:
            return 1
        case _:
            return n * factorial(n - 1)

factorial(5)

In [ ]:
module Factorial =
    let rec calc n = 
        match n with
        // we can use None like Elixer; but F# is more functional in this context
        // we will then have to handle None as well in last match when doing recursion
        // invalidArg "n" "Input must be a non-negative integer."

        | n when n < 0 -> invalidArg "n" "Input must be a non-negative integer"
        // this is equivalent to | n when n < 0 -> raise (System.ArgumentException("Input must be a non-negative integer."))
        
        | 0 -> 1
        | n -> n * calc (n - 1)
        
// Sadly cant use of from Elixer example; because its F# keyword
Factorial.calc 5

In [ ]:
int Factorial(int n) => n switch // to remove squigly we need to cover all cases
{
    0 => 1,
    > 0 => n * Factorial(n - 1),
    // _ => throw new ArgumentException("Input must be a non-negative integer")
};

Factorial(5)

In [ ]:
// Iterations and Linq

using System.Linq;

int Factorial(int n)
{
    if (n < 0) throw new ArgumentException("Input must be a non-negative integer");
    return Enumerable.Range(1, n).Aggregate(1, (acc, x) => acc * x);
}

Factorial(5)

# Resources 📚

- https://www.youtube.com/watch?v=nvVmFqmZX9s The sceptics guide to pattern matching - Matt Ellis - NDC Oslo 2021